In [1]:
import torch

In [2]:
# tensor of (2,3) size
t1 = torch.zeros(size=(2,3), dtype = torch.float32)
t2 = torch.ones(size=(2,3), dtype = torch.float32)
t3 = torch.rand(size=(2,3), dtype = torch.float32)
print(t1,t2,t3)

tensor([[0., 0., 0.],
        [0., 0., 0.]]) tensor([[1., 1., 1.],
        [1., 1., 1.]]) tensor([[0.4503, 0.2002, 0.6652],
        [0.0870, 0.9724, 0.8549]])


In [3]:
# porp of a tensor
print(t2.size(), t2.device, t2.dtype)

torch.Size([2, 3]) cpu torch.float32


In [4]:
import numpy as np

In [5]:
# Transform a tensor to a GPU tensor
# t2 = t2.cuda()

In [6]:
# From NumPy to tensor and viceversa
np_arr = np.random.rand(2,3)
print(np_arr.shape)

(2, 3)


In [7]:
t = torch.from_numpy(np_arr)
t.size()

torch.Size([2, 3])

In [8]:
np_arr_again = t.numpy()
np_arr_again.shape

(2, 3)

## Neural networks and Backpropagation
- Define network architecture
- While Training
    1. Forward pass : feed input and obatain net prediction
    2. Compute loss : compare prediction with the "Ground Truth"(label)
    3. Backward padss : compute gradient of the Network parameters w.r.t to the loss function(Pytorch autograd)
    4. Update the network params(Pytorch Optimizer)
    5. Repeat

### Computational Graph
- Support for forward and backward
- is a directed graph keeping track of all operation performed on Variables
- Nodes represet: 
    1. Variables
    2. Operations
- Pytorch Autograd handled this, providing automatic differentation

In [9]:
from torch.autograd import Variable

In [10]:
# graph crated on the fly Wh h Wx x
x = Variable(torch.rand(1,10))
prev_h = Variable(torch.rand(1,20))
W_h = Variable(torch.rand(20,20))
W_x = Variable(torch.rand(20,10))

### Basic compontens
- Net architecture (torch.nn.Module)
- Dataset (torch.utils.data.Dataset)
- Loss function + optimizer
- Training lopp


## Network form scratch

In [11]:
import torch.nn as nn

In [12]:
class NeuralNet(nn.Module):
    # inizialization of network layer
    def __init__(self,input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    # function descibing Input data path through net
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out



In [13]:
input_size, hidden_size, num_classes = 784, 500, 10
model = NeuralNet(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes)
print(model)

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [14]:
from PIL import Image
import torchvision.transforms.functional as TF

In [15]:
image = Image.open('img/cat.jpg')
cat = TF.to_tensor(image)
cat.unsqueeze_(0)
print(cat.shape)

torch.Size([1, 3, 400, 640])


In [16]:
cat = torch.rand(784)
cat = Variable(cat.view(-1, 28*28))
# cat should be 28*28

In [17]:
forward_res = model(cat)
forward_res

tensor([[-0.1050,  0.0173, -0.1056,  0.0905,  0.0284,  0.0654, -0.2433, -0.2287,
          0.0087, -0.0208]], grad_fn=<AddmmBackward>)

In [18]:
# Dataset
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [19]:
class DatasetMNIST(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        # load image ( Height * Width * Channels)
        # (H, W, C) => (C, H, W)
        image = self.data.iloc[index,1:].values.astype(np.uint8).reshape(1, 28, 28)
        label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image,label
        

In [20]:
transform = transforms.Compose([transforms.ToPILImage(), 
                                transforms.RandomHorizontalFlip(0.5), 
                                transforms.ToTensor()])
train_dataset = DatasetMNIST(file_path='input/train.csv', transform=transform)

In [21]:
import torchvision
# dataloader efficent iterator
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data', train=True,download=True,
                              transform=torchvision.transforms.Compose([
                                  torchvision.transforms.ToTensor(),
                                  torchvision.transforms.Normalize((0.1307,),(0.3081,))
                              ])),
batch_size=100, shuffle=True)

In [22]:
# Loss and Optimizer
loss = nn.CrossEntropyLoss()
input = torch.rand(3,5,requires_grad=True)
target = torch.empty(3,dtype=torch.long).random_(5)
output = loss(input, target) # compute loss
output.backward() # compute gradient backward

In [23]:
import torch.optim as optim
# optimizer update net parameters depending on the gradient of the backward step
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
var1 = torch.rand(5)
var2 = torch.rand(5)
optimizer = optim.Adam([var1,var2],lr=0.0001)

In [24]:
# Training LOOP 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(100):
    tot_loss, tot_samples = 0.0, 0
    for i, data in enumerate(train_loader):
        # Step 1 Batch of input from dataloader
        images, labels = data
        tot_samples += images.size(0)
        
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        # Step 2 zero grading parametes always do before backward()
        optimizer.zero_grad()
        
        # Step 3 get prediction
        outputs = model(images)
        
        # Step 4 compute loss
        loss = criterion(outputs, labels)
        
        # Step 5 compute gradients
        loss.backward()
        
        # Step 6 optimize
        optimizer.step()
        
        #logging
        tot_loss += (loss.item()*images.size(0))
    # End epoch
    print("Epoch %d loss is: %.6f"%(epoch,(tot_loss*1.0/float(tot_samples))))
#End Training

Epoch 0 loss is: 0.303763
Epoch 1 loss is: 0.130319
Epoch 2 loss is: 0.089105
Epoch 3 loss is: 0.066920
Epoch 4 loss is: 0.052757
Epoch 5 loss is: 0.042692
Epoch 6 loss is: 0.034716
Epoch 7 loss is: 0.028133
Epoch 8 loss is: 0.023180
Epoch 9 loss is: 0.019550
Epoch 10 loss is: 0.015843
Epoch 11 loss is: 0.013341
Epoch 12 loss is: 0.011330
Epoch 13 loss is: 0.009884
Epoch 14 loss is: 0.008542
Epoch 15 loss is: 0.007312
Epoch 16 loss is: 0.006317
Epoch 17 loss is: 0.005686
Epoch 18 loss is: 0.005048
Epoch 19 loss is: 0.004473
Epoch 20 loss is: 0.004065
Epoch 21 loss is: 0.003681
Epoch 22 loss is: 0.003383
Epoch 23 loss is: 0.003139
Epoch 24 loss is: 0.002917
Epoch 25 loss is: 0.002713
Epoch 26 loss is: 0.002538
Epoch 27 loss is: 0.002364
Epoch 28 loss is: 0.002247
Epoch 29 loss is: 0.002120
Epoch 30 loss is: 0.002005
Epoch 31 loss is: 0.001893
Epoch 32 loss is: 0.001818
Epoch 33 loss is: 0.001731
Epoch 34 loss is: 0.001662
Epoch 35 loss is: 0.001579
Epoch 36 loss is: 0.001510
Epoch 37 lo

In [25]:
torch.save(model.state_dict(),'./model/fnn_model.pkl')